# Chatting with a customized teachable agent

A customized teachable agent is a personalized assistant who executes and memorizes various tasks. \
In this notebook, we see how this agent memorizes tasks by maintaining an internal context of each task.

## Set your API Endpoint

The [`config_list_from_json`](https://docs.ag2.ai/latest/docs/api-reference/autogen/config_list_from_json/#autogen.config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.customized_teachability import CustomizedTeachability

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": ["gpt-4o", ],
    },
)

print(config_list[0]["model"])

gpt-4o


````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/llm-configuration).
:::
````

## Construct Agents

In [2]:
# Start by instantiating any agent that inherits from ConversableAgent.
teachable_agent = ConversableAgent(
    name="teachable_agent",  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
)

regular_agent = ConversableAgent(
    name="regular_agent",  # The name is flexible, but should not contain spaces to work in group chat.
    llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
)

# Instantiate the Teachability capability. Its parameters are all optional.
customized_teachability = CustomizedTeachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
customized_teachability.add_to_agent(teachable_agent)

# Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
    max_consecutive_auto_reply=0,
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


CLEARING MEMORY


## Regular agent as a baseline
First, we see how a regular agent forgets the context of a task.

Imagine a user who asks for an itinerary of Hawaii.

In [3]:
text = "Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?"

user.initiate_chat(regular_agent, message=text, clear_history=True)

user (to regular_agent):

Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
regular_agent (to user):

Certainly! Planning your first day in Oahu, Hawaii should be both exciting and relaxing. Here's a balanced itinerary, allowing you to experience the island's beauty, culture, and cuisine without feeling rushed:

### Morning

**1. Arrival and Breakfast**
- **Arrival**: Aim to arrive in Oahu early in the morning to maximize your day. If you flew in the day before, even better!
- **Breakfast**: Start with a hearty breakfast at *Azure* in Waikiki, which offers a beachfront setting. Try local specialties like loco moco or a fresh tropical fruit platter.

**2. Explore Waikiki Beach**
- After breakfast, take a leisurely stroll along Waikiki Beach. Depending on your interests, you can relax on the sand or try some water activities like surfing or paddleboarding. Th

ChatResult(chat_id=None, chat_history=[{'content': 'Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?', 'role': 'assistant', 'name': 'user'}, {'content': "Certainly! Planning your first day in Oahu, Hawaii should be both exciting and relaxing. Here's a balanced itinerary, allowing you to experience the island's beauty, culture, and cuisine without feeling rushed:\n\n### Morning\n\n**1. Arrival and Breakfast**\n- **Arrival**: Aim to arrive in Oahu early in the morning to maximize your day. If you flew in the day before, even better!\n- **Breakfast**: Start with a hearty breakfast at *Azure* in Waikiki, which offers a beachfront setting. Try local specialties like loco moco or a fresh tropical fruit platter.\n\n**2. Explore Waikiki Beach**\n- After breakfast, take a leisurely stroll along Waikiki Beach. Depending on your interests, you can relax on the sand or try some water activities like surfing or paddleboarding. The beach has rental shops if you didn’t bring your 

An example reply can be (LLM's actual reply may vary):


Morning **Arrival and Breakfast** ... **Breakfast at Azure in Waikiki** ... **Explore Waikiki Beach** ... \
Midday **Visit Diamond Head State Monument** ... **Lunch** ... \
Afternoon **Explore Honolulu** ... **Ala Moana Center** ... \
Evening **Dinner with a View** ... **Sunset at Waikiki Beach** ... **Nightlife in Waikiki**

--------

Next, the user is satisfied with the schedule but only asks for a small change to include snorkeling. \
However, the agent forgets the context and recommends something completely different. (Assume the user does not manually provide chat history in a session to the agent.)

In [4]:
text = "Can you add snorkeling activities to this Oahu trip because I like snorkeling?"

user.initiate_chat(regular_agent, message=text, clear_history=True)

user (to regular_agent):

Can you add snorkeling activities to this Oahu trip because I like snorkeling?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
regular_agent (to user):

Of course! Here’s an itinerary that includes some snorkeling activities on your Oahu trip. Let me know if you'd like to adjust any part of it.

**Day 1: Arrival in Oahu**
- Arrive at Daniel K. Inouye International Airport.
- Check into your hotel.
- Enjoy a relaxing walk along Waikiki Beach to unwind.

**Day 2: Hanauma Bay Snorkeling**
- Morning: Head to Hanauma Bay Nature Preserve, one of the best snorkeling spots in Oahu. Arrive early, as it’s a popular destination.
- Afternoon: Have a picnic lunch on the beach.
- Evening: Return to Waikiki for dinner at a beachfront restaurant.

**Day 3: North Shore and Shark's Cove**
- Morning: Drive to the North Shore.
- Late Morning: Snorkel at Shark's Cove, which is known for vibrant underwater life and vol

ChatResult(chat_id=None, chat_history=[{'content': 'Can you add snorkeling activities to this Oahu trip because I like snorkeling?', 'role': 'assistant', 'name': 'user'}, {'content': "Of course! Here’s an itinerary that includes some snorkeling activities on your Oahu trip. Let me know if you'd like to adjust any part of it.\n\n**Day 1: Arrival in Oahu**\n- Arrive at Daniel K. Inouye International Airport.\n- Check into your hotel.\n- Enjoy a relaxing walk along Waikiki Beach to unwind.\n\n**Day 2: Hanauma Bay Snorkeling**\n- Morning: Head to Hanauma Bay Nature Preserve, one of the best snorkeling spots in Oahu. Arrive early, as it’s a popular destination.\n- Afternoon: Have a picnic lunch on the beach.\n- Evening: Return to Waikiki for dinner at a beachfront restaurant.\n\n**Day 3: North Shore and Shark's Cove**\n- Morning: Drive to the North Shore.\n- Late Morning: Snorkel at Shark's Cove, which is known for vibrant underwater life and volcanic formations.\n- Afternoon: Enjoy a famou

An example reply below cancels almost all Day-1 activities and discusses the irrelevant Day-2 to Day-7 schedule. \
It is because the agent forgets all context and reschedules a 7-day trip.


**Day 1: Arrival in Oahu** - Arrive at Daniel K. Inouye International Airport. - Check into your hotel. - Enjoy a relaxing walk along Waikiki Beach to unwind. \
**Day 2: Hanauma Bay Snorkeling** - Morning: Head to Hanauma Bay Nature Preserve ... \
... \
**Day 7: ...**

## Customized Teachable Agent

In comparison, our customized teachable agent **memorizes the context** and makes changes accordingly.

In [5]:
text = "Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?"

user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?

--------------------------------------------------------------------------------
teachable_agent (to user):

Certainly! Here’s a suggested plan for the first day of your 5-day trip to Oahu, Hawaii:

**Morning:**

1. **Arrival and Breakfast:**
   - Arrive at Daniel K. Inouye International Airport in Honolulu.
   - Pick up your rental car or arrange for transportation to your hotel.
   - Check into your hotel, if available, or store your luggage.
   - Head to a local breakfast spot like *Koko Head Café* or *Heavenly Island Lifestyle* for a delicious island-style breakfast.

2. **Diamond Head Hike:**
   - Head to Diamond Head State Monument for a hike. It's a popular trail and offers stunning views of Waikiki and the Pacific Ocean. Remember to bring water, sunscreen, and wear comfortable shoes.

**Afternoon:**

3. **Lunch:**
   - Make your way back to Waikiki and enjoy lunch at *Duke’s Waikik

ChatResult(chat_id=None, chat_history=[{'content': 'Would you plan the first day of a 5-day trip to Oahu, Hawaii for me?', 'role': 'assistant', 'name': 'user'}, {'content': "Certainly! Here’s a suggested plan for the first day of your 5-day trip to Oahu, Hawaii:\n\n**Morning:**\n\n1. **Arrival and Breakfast:**\n   - Arrive at Daniel K. Inouye International Airport in Honolulu.\n   - Pick up your rental car or arrange for transportation to your hotel.\n   - Check into your hotel, if available, or store your luggage.\n   - Head to a local breakfast spot like *Koko Head Café* or *Heavenly Island Lifestyle* for a delicious island-style breakfast.\n\n2. **Diamond Head Hike:**\n   - Head to Diamond Head State Monument for a hike. It's a popular trail and offers stunning views of Waikiki and the Pacific Ocean. Remember to bring water, sunscreen, and wear comfortable shoes.\n\n**Afternoon:**\n\n3. **Lunch:**\n   - Make your way back to Waikiki and enjoy lunch at *Duke’s Waikiki*, located right

An example reply is:

Morning: **Arrival and Breakfast:** ... **Koko Head Café or Heavenly Island Lifestyle for breakfast.** ...  **Diamond Head Hike:** ... \
Afternoon: **Lunch:** ... **Beach Time:** ... **Explore Waikiki:** ... \
Evening: **Dinner and Sunset:** ... **Nightlife or Relax:** ... 
  
-----------


In [6]:
text = "Can you add snorkeling activities to this Oahu trip because I like snorkeling?"

user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Can you add snorkeling activities to this Oahu trip because I like snorkeling?

--------------------------------------------------------------------------------


Add of existing embedding ID: 1
Insert of existing embedding ID: 1


teachable_agent (to user):

Certainly! Let's add some snorkeling activities to your first day in Oahu. Here's an updated itinerary:

**Morning:**

1. **Arrival and Breakfast:**
   - Arrive at Daniel K. Inouye International Airport in Honolulu.
   - Pick up your rental car or arrange for transportation to your hotel.
   - Check into your hotel, if available, or store your luggage.
   - Head to a local breakfast spot like *Koko Head Café* or *Heavenly Island Lifestyle* for a delicious island-style breakfast.

2. **Diamond Head Hike:**
   - Head to Diamond Head State Monument for a hike. It's a popular trail and offers stunning views of Waikiki and the Pacific Ocean. Remember to bring water, sunscreen, and wear comfortable shoes.

**Afternoon:**

3. **Snorkeling at Hanauma Bay:**
   - After your hike, head over to Hanauma Bay Nature Preserve for a snorkeling adventure. It's one of the best spots on the island to see a variety of marine life. Be sure to check the preserve's schedule and wa

ChatResult(chat_id=None, chat_history=[{'content': 'Can you add snorkeling activities to this Oahu trip because I like snorkeling?', 'role': 'assistant', 'name': 'user'}, {'content': "Certainly! Let's add some snorkeling activities to your first day in Oahu. Here's an updated itinerary:\n\n**Morning:**\n\n1. **Arrival and Breakfast:**\n   - Arrive at Daniel K. Inouye International Airport in Honolulu.\n   - Pick up your rental car or arrange for transportation to your hotel.\n   - Check into your hotel, if available, or store your luggage.\n   - Head to a local breakfast spot like *Koko Head Café* or *Heavenly Island Lifestyle* for a delicious island-style breakfast.\n\n2. **Diamond Head Hike:**\n   - Head to Diamond Head State Monument for a hike. It's a popular trail and offers stunning views of Waikiki and the Pacific Ocean. Remember to bring water, sunscreen, and wear comfortable shoes.\n\n**Afternoon:**\n\n3. **Snorkeling at Hanauma Bay:**\n   - After your hike, head over to Hanau

Our customized teachable agent **memorizes the context** and changes only one activity to snorkeling, like below:

Morning: Arrival and Breakfast: ... Koko Head Café or Heavenly Island Lifestyle for breakfast. ...  Diamond Head Hike: ... \
Afternoon: **Snorkeling at Hanauma Bay:** ... Lunch: ... Beach Time: ... \
Evening: Dinner and Sunset: ... Nightlife or Relax: ... 
  
-----------


## Context Switch

Our customized teachable agent maintains multiple tasks and their contexts, so it can switch context. \
For example, we can switch to an email writing task and then come back to the Hawaii scheduling task.

In [7]:
text = "Would you write an email to thank Mike for organizing the meet-up event for team members around US to get together on 5/1 at Boston?"

user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Would you write an email to thank Mike for organizing the meet-up event for team members around US to get together on 5/1 at Boston?

--------------------------------------------------------------------------------
teachable_agent (to user):

Certainly! Here's a draft for your email:

---

Subject: Thank You for Organizing the Team Meet-Up in Boston!

Hi Mike,

I hope this message finds you well. I wanted to extend a heartfelt thank you for organizing the meet-up event for our team members from across the US on May 1st in Boston. Your efforts in bringing everyone together did not go unnoticed.

The event was a fantastic opportunity for us all to connect face-to-face and strengthen our team bonds. I truly appreciate the time and effort you put into making this happen. From arranging the venue to coordinating all the details, everything was seamless and contributed to a successful and enjoyable gathering.

Thank you once again for your hard work and dedication

ChatResult(chat_id=None, chat_history=[{'content': 'Would you write an email to thank Mike for organizing the meet-up event for team members around US to get together on 5/1 at Boston?', 'role': 'assistant', 'name': 'user'}, {'content': "Certainly! Here's a draft for your email:\n\n---\n\nSubject: Thank You for Organizing the Team Meet-Up in Boston!\n\nHi Mike,\n\nI hope this message finds you well. I wanted to extend a heartfelt thank you for organizing the meet-up event for our team members from across the US on May 1st in Boston. Your efforts in bringing everyone together did not go unnoticed.\n\nThe event was a fantastic opportunity for us all to connect face-to-face and strengthen our team bonds. I truly appreciate the time and effort you put into making this happen. From arranging the venue to coordinating all the details, everything was seamless and contributed to a successful and enjoyable gathering.\n\nThank you once again for your hard work and dedication. Looking forward to 

In [8]:
text = "Can you change the breakfast restaurant of Day 1 at Oahu?"

user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachable_agent):

Can you change the breakfast restaurant of Day 1 at Oahu?

--------------------------------------------------------------------------------


Add of existing embedding ID: 1
Insert of existing embedding ID: 1


teachable_agent (to user):

Sure! Let's update the breakfast restaurant on Day 1 of your Oahu itinerary. Here’s the revised entry for your breakfast:

**Morning:**

1. **Arrival and Breakfast:**
   - Arrive at Daniel K. Inouye International Airport in Honolulu.
   - Pick up your rental car or arrange for transportation to your hotel.
   - Check into your hotel, if available, or store your luggage.
   - Head to a local breakfast spot like *Island Vintage Coffee* or *Over Easy* to kick off your day with a fantastic breakfast.

Let me know if there's anything else you'd like to adjust!

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (73a40586-c31b-4aff-934d-4fe7eee54e21): Maximum number of consecutive auto-replies reached


ChatResult(chat_id=None, chat_history=[{'content': 'Can you change the breakfast restaurant of Day 1 at Oahu?', 'role': 'assistant', 'name': 'user'}, {'content': "Sure! Let's update the breakfast restaurant on Day 1 of your Oahu itinerary. Here’s the revised entry for your breakfast:\n\n**Morning:**\n\n1. **Arrival and Breakfast:**\n   - Arrive at Daniel K. Inouye International Airport in Honolulu.\n   - Pick up your rental car or arrange for transportation to your hotel.\n   - Check into your hotel, if available, or store your luggage.\n   - Head to a local breakfast spot like *Island Vintage Coffee* or *Over Easy* to kick off your day with a fantastic breakfast.\n\nLet me know if there's anything else you'd like to adjust!", 'role': 'user', 'name': 'teachable_agent'}], summary="Sure! Let's update the breakfast restaurant on Day 1 of your Oahu itinerary. Here’s the revised entry for your breakfast:\n\n**Morning:**\n\n1. **Arrival and Breakfast:**\n   - Arrive at Daniel K. Inouye Inter

Our customized teachable agent **memorizes the context** again and changes only the breakfast restaurant, like below:

Morning: Arrival and Breakfast: ... **Island Vintage Coffee or Over Easy for breakfast.** ...  \
  
-----------